# Рекуррентные нейронные сети

В предыдущем модуле мы рассмотрели богатые семантические представления текста. Архитектура, которую мы использовали, захватывает агрегированное значение слов в предложении, но не учитывает **порядок** слов, поскольку операция агрегации, следующая за эмбеддингами, удаляет эту информацию из исходного текста. Поскольку эти модели не могут представлять порядок слов, они не способны решать более сложные или неоднозначные задачи, такие как генерация текста или ответы на вопросы.

Чтобы уловить смысл последовательности текста, мы будем использовать архитектуру нейронной сети, называемую **рекуррентной нейронной сетью** или RNN. При использовании RNN мы пропускаем наше предложение через сеть по одному токену за раз, и сеть создает некоторое **состояние**, которое затем передается в сеть вместе со следующим токеном.

![Изображение, показывающее пример генерации рекуррентной нейронной сети.](../../../../../translated_images/ru/rnn.27f5c29c53d727b5.webp)

Учитывая входную последовательность токенов $X_0,\dots,X_n$, RNN создает последовательность блоков нейронной сети и обучает эту последовательность от начала до конца с использованием обратного распространения ошибки. Каждый блок сети принимает пару $(X_i,S_i)$ в качестве входных данных и производит $S_{i+1}$ в качестве результата. Финальное состояние $S_n$ или выход $Y_n$ передается в линейный классификатор для получения результата. Все блоки сети имеют одинаковые веса и обучаются от начала до конца за один проход обратного распространения.

> На рисунке выше показана рекуррентная нейронная сеть в развернутой форме (слева) и в более компактной рекуррентной форме (справа). Важно понимать, что все ячейки RNN имеют одинаковые **общие веса**.

Поскольку векторы состояния $S_0,\dots,S_n$ передаются через сеть, RNN способна обучаться последовательным зависимостям между словами. Например, когда слово *not* появляется где-то в последовательности, сеть может научиться отрицать определенные элементы внутри вектора состояния.

Внутри каждой ячейки RNN содержатся две матрицы весов: $W_H$ и $W_I$, а также смещение $b$. На каждом шаге RNN, учитывая вход $X_i$ и входное состояние $S_i$, выходное состояние вычисляется как $S_{i+1} = f(W_H\times S_i + W_I\times X_i+b)$, где $f$ — это функция активации (часто $\tanh$).

> Для задач, таких как генерация текста (которую мы рассмотрим в следующем разделе) или машинный перевод, мы также хотим получать некоторое выходное значение на каждом шаге RNN. В этом случае существует еще одна матрица $W_O$, и выход вычисляется как $Y_i=f(W_O\times S_i+b_O)$.

Давайте посмотрим, как рекуррентные нейронные сети могут помочь нам классифицировать наш набор данных новостей.

> Для песочницы необходимо выполнить следующую ячейку, чтобы убедиться, что нужная библиотека установлена, а данные предварительно загружены. Если вы работаете локально, этот шаг можно пропустить.


In [1]:
import sys
!{sys.executable} -m pip install --quiet tensorflow_datasets==4.4.0
!cd ~ && wget -q -O - https://mslearntensorflowlp.blob.core.windows.net/data/tfds-ag-news.tgz | tar xz

In [2]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

# We are going to be training pretty large models. In order not to face errors, we need
# to set tensorflow option to grow GPU memory allocation when required
physical_devices = tf.config.list_physical_devices('GPU') 
if len(physical_devices)>0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

ds_train, ds_test = tfds.load('ag_news_subset').values()

При обучении крупных моделей распределение памяти GPU может стать проблемой. Также может понадобиться экспериментировать с различными размерами минибатчей, чтобы данные помещались в память GPU, а обучение проходило достаточно быстро. Если вы запускаете этот код на своей машине с GPU, попробуйте настроить размер минибатча для ускорения обучения.

> **Примечание**: Известно, что некоторые версии драйверов NVidia не освобождают память после обучения модели. В этом ноутбуке мы запускаем несколько примеров, и это может привести к исчерпанию памяти в определённых конфигурациях, особенно если вы проводите собственные эксперименты в рамках того же ноутбука. Если вы столкнулись с странными ошибками при запуске обучения модели, попробуйте перезапустить ядро ноутбука.


In [3]:
batch_size = 16
embed_size = 64

## Простой классификатор на основе RNN

В случае простой RNN каждая рекуррентная единица представляет собой простую линейную сеть, которая принимает на вход вектор входных данных и вектор состояния, а затем выдает новый вектор состояния. В Keras это можно реализовать с помощью слоя `SimpleRNN`.

Хотя мы можем передавать токены в формате one-hot кодирования напрямую в слой RNN, это не лучшая идея из-за их высокой размерности. Поэтому мы будем использовать слой embedding для уменьшения размерности векторов слов, затем слой RNN, а в конце — классификатор `Dense`.

> **Note**: В случаях, когда размерность не слишком велика, например, при использовании токенизации на уровне символов, может быть целесообразно передавать токены в формате one-hot кодирования напрямую в ячейку RNN.


In [4]:
vocab_size = 20000

vectorizer = keras.layers.experimental.preprocessing.TextVectorization(
    max_tokens=vocab_size,
    input_shape=(1,))

model = keras.models.Sequential([
    vectorizer,
    keras.layers.Embedding(vocab_size, embed_size),
    keras.layers.SimpleRNN(16),
    keras.layers.Dense(4,activation='softmax')
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_vectorization (TextVect (None, None)              0         
_________________________________________________________________
embedding (Embedding)        (None, None, 64)          1280000   
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 16)                1296      
_________________________________________________________________
dense (Dense)                (None, 4)                 68        
Total params: 1,281,364
Trainable params: 1,281,364
Non-trainable params: 0
_________________________________________________________________


> **Примечание:** Здесь мы используем необученный слой встраивания для упрощения, но для достижения лучших результатов можно использовать предварительно обученный слой встраивания с помощью Word2Vec, как описано в предыдущем разделе. Это будет хорошим упражнением — адаптировать этот код для работы с предварительно обученными встраиваниями.

Теперь давайте обучим нашу RNN. В целом, RNN довольно сложно обучать, так как после разворачивания ячеек RNN вдоль длины последовательности количество слоев, участвующих в обратном распространении ошибки, становится очень большим. Поэтому необходимо выбрать меньшую скорость обучения и обучать сеть на большем наборе данных, чтобы получить хорошие результаты. Это может занять довольно много времени, поэтому предпочтительно использовать GPU.

Чтобы ускорить процесс, мы будем обучать модель RNN только на заголовках новостей, исключая описание. Вы можете попробовать обучить модель с использованием описания и посмотреть, удастся ли вам добиться её обучения.


In [5]:
def extract_title(x):
    return x['title']

def tupelize_title(x):
    return (extract_title(x),x['label'])

print('Training vectorizer')
vectorizer.adapt(ds_train.take(2000).map(extract_title))

Training vectorizer


In [6]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize_title).batch(batch_size),validation_data=ds_test.map(tupelize_title).batch(batch_size))

7500/7500 [==============================] - 82s 11ms/step - loss: 0.6629 - acc: 0.7623 - val_loss: 0.5559 - val_acc: 0.7995


> **Примечание**: точность здесь, вероятно, будет ниже, потому что мы обучаемся только на заголовках новостей.


## Повторное рассмотрение последовательностей переменной длины

Помните, что слой `TextVectorization` автоматически дополняет последовательности переменной длины в минибатче токенами-заполнителями. Оказывается, эти токены также участвуют в обучении, и это может усложнить сходимость модели.

Существует несколько подходов, которые мы можем использовать, чтобы минимизировать количество дополнений. Один из них — это упорядочить набор данных по длине последовательностей и сгруппировать все последовательности по размеру. Это можно сделать с помощью функции `tf.data.experimental.bucket_by_sequence_length` (см. [документацию](https://www.tensorflow.org/api_docs/python/tf/data/experimental/bucket_by_sequence_length)).

Другой подход — использовать **маскирование**. В Keras некоторые слои поддерживают дополнительный ввод, который указывает, какие токены следует учитывать при обучении. Чтобы включить маскирование в нашу модель, мы можем либо добавить отдельный слой `Masking` ([документация](https://keras.io/api/layers/core_layers/masking/)), либо указать параметр `mask_zero=True` в нашем слое `Embedding`.

> **Примечание**: Обучение займет около 5 минут для завершения одной эпохи на всем наборе данных. Вы можете прервать обучение в любой момент, если у вас закончится терпение. Также вы можете ограничить объем данных, используемых для обучения, добавив оператор `.take(...)` после наборов данных `ds_train` и `ds_test`.


In [7]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

model = keras.models.Sequential([
    vectorizer,
    keras.layers.Embedding(vocab_size,embed_size,mask_zero=True),
    keras.layers.SimpleRNN(16),
    keras.layers.Dense(4,activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

7500/7500 [==============================] - 371s 49ms/step - loss: 0.5401 - acc: 0.8079 - val_loss: 0.3780 - val_acc: 0.8822


Теперь, когда мы используем маскирование, мы можем обучить модель на всем наборе данных заголовков и описаний.

> **Примечание**: Вы заметили, что мы использовали векторизатор, обученный на заголовках новостей, а не на полном тексте статьи? Потенциально это может привести к игнорированию некоторых токенов, поэтому лучше переобучить векторизатор. Однако эффект может быть совсем незначительным, поэтому для упрощения мы будем использовать ранее обученный векторизатор.


## LSTM: Долговременная кратковременная память

Одна из главных проблем рекуррентных нейронных сетей (RNN) — это **затухающие градиенты**. RNN могут быть довольно длинными, и им может быть сложно передавать градиенты обратно к первому слою сети во время обратного распространения. Когда это происходит, сеть не может обучаться выявлению связей между удаленными токенами. Один из способов избежать этой проблемы — ввести **явное управление состоянием** с помощью **гейтов**. Две наиболее распространенные архитектуры, использующие гейты, — это **долговременная кратковременная память** (LSTM) и **гейтированная релейная единица** (GRU). Здесь мы рассмотрим LSTM.

![Изображение, показывающее пример ячейки долговременной кратковременной памяти](../../../../../lessons/5-NLP/16-RNN/images/long-short-term-memory-cell.svg)

Сеть LSTM организована аналогично RNN, но есть два состояния, которые передаются от слоя к слою: фактическое состояние $c$ и скрытый вектор $h$. В каждом блоке скрытый вектор $h_{t-1}$ комбинируется с входом $x_t$, и вместе они управляют тем, что происходит с состоянием $c_t$ и выходом $h_{t}$ через **гейты**. Каждый гейт имеет сигмоидную активацию (выход в диапазоне $[0,1]$), которую можно рассматривать как побитовую маску при умножении на вектор состояния. LSTM имеют следующие гейты (слева направо на изображении выше):
* **гейт забывания**, который определяет, какие компоненты вектора $c_{t-1}$ нужно забыть, а какие пропустить дальше.
* **входной гейт**, который определяет, сколько информации из входного вектора и предыдущего скрытого вектора следует включить в вектор состояния.
* **выходной гейт**, который берет новый вектор состояния и решает, какие его компоненты будут использованы для формирования нового скрытого вектора $h_t$.

Компоненты состояния $c$ можно рассматривать как флаги, которые можно включать и выключать. Например, когда мы встречаем имя *Алиса* в последовательности, мы предполагаем, что речь идет о женщине, и поднимаем флаг в состоянии, который говорит, что в предложении есть женский существительный. Когда мы далее встречаем слова *и Том*, мы поднимаем флаг, который говорит, что у нас есть множественное число существительных. Таким образом, манипулируя состоянием, мы можем отслеживать грамматические свойства предложения.

> **Note**: Вот отличный ресурс для понимания внутреннего устройства LSTM: [Understanding LSTM Networks](https://colah.github.io/posts/2015-08-Understanding-LSTMs/) от Кристофера Олаха.

Хотя внутренняя структура ячейки LSTM может выглядеть сложной, Keras скрывает эту реализацию внутри слоя `LSTM`, поэтому единственное, что нам нужно сделать в приведенном выше примере, — это заменить рекуррентный слой:


In [8]:
model = keras.models.Sequential([
    vectorizer,
    keras.layers.Embedding(vocab_size, embed_size),
    keras.layers.LSTM(8),
    keras.layers.Dense(4,activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(8),validation_data=ds_test.map(tupelize).batch(8))

15000/15000 [==============================] - 188s 13ms/step - loss: 0.5692 - acc: 0.7916 - val_loss: 0.3441 - val_acc: 0.8870


## Двунаправленные и многослойные RNN

В наших предыдущих примерах рекуррентные сети обрабатывали последовательности от начала до конца. Это кажется естественным, так как соответствует направлению, в котором мы читаем или слушаем речь. Однако в сценариях, где требуется произвольный доступ к элементам входной последовательности, логичнее выполнять рекуррентные вычисления в обоих направлениях. RNN, которые позволяют вычисления в двух направлениях, называются **двунаправленными** (bidirectional) RNN, и их можно создать, обернув рекуррентный слой в специальный слой `Bidirectional`.

> **Note**: Слой `Bidirectional` создает две копии слоя внутри себя и устанавливает свойство `go_backwards` одной из этих копий в значение `True`, что заставляет её обрабатывать последовательность в обратном направлении.

Рекуррентные сети, будь то однонаправленные или двунаправленные, захватывают закономерности внутри последовательности и сохраняют их в векторах состояния или возвращают их в качестве результата. Как и в случае с сверточными сетями, мы можем добавить еще один рекуррентный слой после первого, чтобы захватывать закономерности более высокого уровня, построенные на основе закономерностей более низкого уровня, извлеченных первым слоем. Это приводит нас к понятию **многослойной RNN**, которая состоит из двух или более рекуррентных сетей, где выход предыдущего слоя передается в следующий слой в качестве входных данных.

![Изображение, показывающее многослойную LSTM-RNN](../../../../../translated_images/ru/multi-layer-lstm.dd975e29bb2a59fe.webp)

*Изображение из [этой замечательной статьи](https://towardsdatascience.com/from-a-lstm-cell-to-a-multilayer-lstm-network-with-pytorch-2899eb5696f3) Фернандо Лопеса.*

Keras упрощает создание таких сетей, так как вам нужно просто добавить больше рекуррентных слоев в модель. Для всех слоев, кроме последнего, необходимо указать параметр `return_sequences=True`, чтобы слой возвращал все промежуточные состояния, а не только финальное состояние рекуррентных вычислений.

Давайте создадим двухслойную двунаправленную LSTM для нашей задачи классификации.

> **Note** Этот код снова выполняется довольно долго, но он дает наивысшую точность из всех, которые мы видели до сих пор. Так что, возможно, стоит подождать и посмотреть результат.


In [9]:
model = keras.models.Sequential([
    vectorizer,
    keras.layers.Embedding(vocab_size, 128, mask_zero=True),
    keras.layers.Bidirectional(keras.layers.LSTM(64,return_sequences=True)),
    keras.layers.Bidirectional(keras.layers.LSTM(64)),    
    keras.layers.Dense(4,activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(batch_size),
          validation_data=ds_test.map(tupelize).batch(batch_size))

5045/7500 [===================>..........] - ETA: 2:33 - loss: 0.3709 - acc: 0.8706

## RNN для других задач

До этого момента мы сосредоточились на использовании RNN для классификации текстовых последовательностей. Однако они могут справляться с множеством других задач, таких как генерация текста и машинный перевод — эти задачи мы рассмотрим в следующем разделе.



---

**Отказ от ответственности**:  
Этот документ был переведен с помощью сервиса автоматического перевода [Co-op Translator](https://github.com/Azure/co-op-translator). Несмотря на наши усилия обеспечить точность, автоматические переводы могут содержать ошибки или неточности. Оригинальный документ на его исходном языке следует считать авторитетным источником. Для получения критически важной информации рекомендуется профессиональный перевод человеком. Мы не несем ответственности за любые недоразумения или неправильные интерпретации, возникшие в результате использования данного перевода.
